In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [14]:
# Load data
df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')  


In [15]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
print("Dataset Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

Missing Values:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687

In [18]:
# 2. Handling missing values
# Filling 'Age' with median
df['Age'] = df['Age'].fillna(df['Age'].median())

# Filling 'Embarked' with mode
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Dropping 'Cabin' due to high number of missing values
df.drop('Cabin', axis=1, inplace=True)

# Verifying missing values after handling
print("\nMissing Values After Handling:")
print(df.isnull().sum())



Missing Values After Handling:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [19]:
# 3. Converting categorical features to numerical
# Label encoding for 'Sex'
le_sex = LabelEncoder()
df['Sex'] = le_sex.fit_transform(df['Sex'])

# One-hot encoding for 'Embarked'
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')

# Dropping non-numeric columns not needed for ML
df.drop(['Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)

# 4. Normalizing/standardizing numerical features
scaler = StandardScaler()
numerical_cols = ['Age', 'Fare', 'SibSp', 'Parch']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])



In [20]:
# 5. Visualizing and removing outliers
# Creating boxplots for numerical columns
plt.figure(figsize=(12, 6))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot of {col}')
plt.tight_layout()
plt.savefig('boxplots_before_outlier_removal.png')
plt.close()

# Removing outliers using IQR method
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# Creating boxplots after outlier removal
plt.figure(figsize=(12, 6))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot of {col} After Outlier Removal')
plt.tight_layout()
plt.savefig('boxplots_after_outlier_removal.png')
plt.close()

# Saving the cleaned dataset
df.to_csv('Titanic-Cleaned-Dataset.csv', index=False)

# Displaying final dataset info
print("\nFinal Dataset Info:")
print(df.info())
print("\nFirst 5 Rows of Cleaned Dataset:")
print(df.head())


Final Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    577 non-null    int64  
 1   Pclass      577 non-null    int64  
 2   Sex         577 non-null    int64  
 3   Age         577 non-null    float64
 4   SibSp       577 non-null    float64
 5   Parch       577 non-null    float64
 6   Fare        577 non-null    float64
 7   Embarked_C  577 non-null    bool   
 8   Embarked_Q  577 non-null    bool   
 9   Embarked_S  577 non-null    bool   
dtypes: bool(3), float64(4), int64(3)
memory usage: 37.8 KB
None

First 5 Rows of Cleaned Dataset:
   Survived  Pclass  Sex       Age     SibSp     Parch      Fare  Embarked_C  \
0         0       3    1 -0.565736  0.432793 -0.473674 -0.502445       False   
2         1       3    0 -0.258337 -0.474545 -0.473674 -0.488854       False   
3         1       1    0  0.433312  0.432793 -0.